# Functionality 4

In [22]:
import pandas as pd
import os
import time
import networkx as nx
import trees as t

In [23]:
# Non usarla
def create_txt_fun4(start1, stop1, start2, stop2):
    for f in ['a2q', 'c2a', 'c2q']:
        fn = 'sx-stackoverflow-' + f + '.txt'
        df = pd.read_csv(fn, sep=' ')
        df.columns = ['u', 'v', 'timestamp']
        
        cond_1 = (df['timestamp'] >= start1) & (df["timestamp"] <= stop1)
        sub_data = df.loc[cond_1, :]
        sub_data.to_csv(f+'.txt', sep=',', index=False, header=False)
        
        cond_2 = (df['timestamp'] >= start2) & (df["timestamp"] <= stop2)
        sub_data = df.loc[cond_2, :]
        sub_data.to_csv(f+'.txt', sep=',', index=False, header=False)

In [24]:
# Non usarla
def build_fun4(start1, stop1, start2, stop2):
    
    start1 = time.mktime(time.strptime(start1, '%Y-%m-%d'))
    stop1 = time.mktime(time.strptime(stop1, '%Y-%m-%d'))
    
    start2 = time.mktime(time.strptime(start2, '%Y-%m-%d'))
    stop2 = time.mktime(time.strptime(stop2, '%Y-%m-%d'))

    # if check():  # If there aren't txt files it calls the function to create them
    create_txt_fun4(start1, stop1, start2, stop2)

    G = nx.DiGraph()  # Initialize graph
    w = 4
    for el in ['a2q', 'c2a', 'c2q']:
        # filter_dataset(el, start, stop)
        w /= 2  # It halves the weight each time it opens a new file
        with open(el+'.txt') as f:
            for line in f:
                u, v, _ = map(int, line.split(','))
                if u != v:
                    if (u, v) in G.edges:
                        tmp_w = G.get_edge_data(u, v)['weight']
                        G.add_edge(u, v, weight=w+tmp_w)
                    else:
                        G.add_edge(u, v, weight=w)
    return G

In [25]:
def create_check_unique_nodes(start_1, end_1, start_2, end_2, user_1, user_2):
    check = True
    # create the two graphs for the intervals time in input
    # Non usare le successive due righe, usane una solo per generare direttamente il grafo unito
    graph_1 = t.build("a", start_1, end_1)
    graph_2 = t.build("a", start_2, end_2)
    
    # collecting the nodes in the two graphs
    nodes_1 = list(graph_1.nodes())
    nodes_2 = list(graph_2.nodes())
    # collect all the nodes in the two different graphs
    nodes_1_2 = nodes_1 + nodes_2
    # check wether the user_1 node in input is only in the graph_1
    # check wether the user_2 node in input is only in the graph_2
    if (user_1 in nodes_2) or (user_2 in nodes_1):
        check = False
    return check

In [26]:
def bfs(graph, user_1, user_2):
    # initialize the predecessor dictionary to keep track of the path
    visisted, predecessor, path = list(), dict(), list()
    find = False
    
    # define the queue list and initizalize queue and visited with user_1 node (our souce)
    queue = [user_1] 
    visited.append(user_1)
    # loop untile the queue is empty
    while queue != []:
        # extract the first element from the queue 
        v = queue.pop(0)
        # considering all the neighbours of the node v
        for neighbour in graph.neighbors(v):
            # if the neighbour has not been visited then we update the visited and queue list and predecessor dictionary
            if  (neighbour not in visited) and (graph[v][neighbour]["weigth"] > 0):
                visited.append(neighbour)
                queue.append(neighbour)
                predecessor[neighbour] = v
                # if finally the neighbour correspond to the target (i.e. user_2) we reconstruct the path using predecessor
                if neighbour == user_2:
                    last = user_2
                    path = list([last])
                    while last != user_1:
                        path.append(predecessor[last])
                        last = predecessor[last]
                    find = True
                    break
    return find, predecessor, path

In [27]:
def FordFulkerson(graph, user_1, user_2):
    
    find, predecessor = bfs(graph, user_1, user_2)
    
    while find:
        path_flow = math.inf
        last = user_2
        
        while last != user_1:
            path_flow = min(path_flow, graph[predecessor[last]][last]["weight"])
            target = predecessor[last]
            
        target = user_2
        while target != user_1:
            source = predecessor[target]
            graph[source][target]["weight"] -= path_flow
            target = source
            
    min_edges = 0
    for (i,j) in graph.edges:
        if (graph[i][j]["weigth"] == 0) and (i in path) and (j in path):
            min_edges += 1
    return min_edges        

In [29]:
def functionality_4(start_1, end_1, start_2, end_2, user_1, user_2):
    # check if the the nodes are unique for the two separated graphs
    check = create_check_unique_nodes(start_1, end_1, start_2, end_2, user_1, user_2)
    if check == False:
        return "The nodes in input should be unique for each graph"
    # create the graph considering two different intervals of time
    graph = build_fun4(start_1, end_1, start_2, end_2)
    #using FordFulkerson function to find the min_edges
    min_edges = FordFulkerson(graph, user_1, user_2)
    return min_edges

In [ ]:
print(functionality_4("2016-01-01", "2016-01-02", "2016-01-03", "2016-01-04", user_1, user_2))